In [1]:
import pystac_client
import xarray as xr

from mccn.loader import stac_load_raster
from mccn.utils import merge_frames, point_data_to_xarray
from tests.utils import generate_test_point_data, get_geobox

In [2]:
frame1 = generate_test_point_data(
    x=(138.62862153159546, 138.6295039082798),
    y=(-34.96915662834354, -34.968540581295116),
    shape=10,
    fill_pct=0.1,
    t=("2020-01-01", "2020-01-05"),
    shape_t=3,
)

frame2 = generate_test_point_data(
    x=(138.62862153159546, 138.6295039082798),
    y=(-34.96915662834354, -34.968540581295116),
    shape=10,
    fill_pct=0.1,
    t=("2020-01-01", "2020-01-05"),
    shape_t=3,
)

gbox = get_geobox([frame1, frame2], 4326, 10, 10)

In [3]:
merged = merge_frames([frame1, frame2], gbox, x_col="X", y_col="Y", t_col="T")
point_ds = point_data_to_xarray(merged, gbox, x_col="X", y_col="Y")
point_ds

<xarray.Dataset> Size: 5kB
Dimensions:      (T: 3, Y: 10, X: 10)
Coordinates:
  * T            (T) datetime64[ns] 24B 2020-01-01 2020-01-03 2020-01-05
    spatial_ref  int32 4B 4326
  * Y            (Y) float64 80B -34.97 -34.97 -34.97 ... -34.97 -34.97 -34.97
  * X            (X) float64 80B 138.6 138.6 138.6 138.6 ... 138.6 138.6 138.6
Data variables:
    radiation    (T, Y, X) float64 2kB nan nan nan nan nan ... nan nan nan nan
    rainfall     (T, Y, X) float64 2kB nan nan nan nan nan ... nan nan nan nan

In [4]:
client = pystac_client.Client.open("http://115.146.84.224:8082")
item1 = client.get_item("item_1")

In [5]:
raster_ds = stac_load_raster([item1], gbox, x_col="X", y_col="Y", t_col="T")

In [6]:
raster_ds

<xarray.Dataset> Size: 472B
Dimensions:      (Y: 10, X: 10, T: 1)
Coordinates:
  * Y            (Y) float64 80B -34.97 -34.97 -34.97 ... -34.97 -34.97 -34.97
  * X            (X) float64 80B 138.6 138.6 138.6 138.6 ... 138.6 138.6 138.6
    spatial_ref  int32 4B 4326
  * T            (T) datetime64[ns] 8B 2024-07-15T06:00:18.830458
Data variables:
    red          (T, Y, X) uint8 100B 0 0 0 0 0 0 0 0 ... 29 240 46 46 143 73 56
    green        (T, Y, X) uint8 100B 0 0 0 0 0 0 0 0 ... 25 221 44 59 173 72 74
    blue         (T, Y, X) uint8 100B 0 0 0 0 0 0 0 ... 24 227 55 41 161 104 94

In [7]:
point_ds

<xarray.Dataset> Size: 5kB
Dimensions:      (T: 3, Y: 10, X: 10)
Coordinates:
  * T            (T) datetime64[ns] 24B 2020-01-01 2020-01-03 2020-01-05
    spatial_ref  int32 4B 4326
  * Y            (Y) float64 80B -34.97 -34.97 -34.97 ... -34.97 -34.97 -34.97
  * X            (X) float64 80B 138.6 138.6 138.6 138.6 ... 138.6 138.6 138.6
Data variables:
    radiation    (T, Y, X) float64 2kB nan nan nan nan nan ... nan nan nan nan
    rainfall     (T, Y, X) float64 2kB nan nan nan nan nan ... nan nan nan nan

In [8]:
merged = xr.merge([point_ds, raster_ds])

In [11]:
merged

<xarray.Dataset> Size: 11kB
Dimensions:      (T: 4, Y: 10, X: 10)
Coordinates:
  * T            (T) datetime64[ns] 32B 2020-01-01 ... 2024-07-15T06:00:18.83...
  * Y            (Y) float64 80B -34.97 -34.97 -34.97 ... -34.97 -34.97 -34.97
  * X            (X) float64 80B 138.6 138.6 138.6 138.6 ... 138.6 138.6 138.6
    spatial_ref  int32 4B 4326
Data variables:
    radiation    (T, Y, X) float64 3kB nan nan nan nan nan ... nan nan nan nan
    rainfall     (T, Y, X) float64 3kB nan nan nan nan nan ... nan nan nan nan
    red          (T, Y, X) float32 2kB nan nan nan nan ... 46.0 143.0 73.0 56.0
    green        (T, Y, X) float32 2kB nan nan nan nan ... 59.0 173.0 72.0 74.0
    blue         (T, Y, X) float32 2kB nan nan nan nan ... 41.0 161.0 104.0 94.0

In [10]:
point_ds.dims, raster_ds.dims

(FrozenMappingWarningOnValuesAccess({'T': 3, 'Y': 10, 'X': 10}),
 FrozenMappingWarningOnValuesAccess({'Y': 10, 'X': 10, 'T': 1}))